In [663]:
%load_ext autoreload
%autoreload 2
# Supress default INFO logging  (Hide Info)

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import logging
import collections
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open 
import pandas as pd
import numpy as np
from numpy import random
from collections import Counter
import heapq #check n largest
import json
import warnings
from sqlalchemy import create_engine 
import spacy
import nltk
import seaborn as sns

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [ ]:
bath_path = r''

computer_security_and_networks = json.load(open(bath_path + r'computer-security-and-networks.json', encoding='utf8'))
algorithms = json.load(open(bath_path + r'algorithms.json', encoding='utf8'))
cloud_computing = json.load(open(bath_path + r'cloud-computing.json', encoding='utf8'))
data_management = json.load(open(bath_path + r'data-management.json', encoding='utf8'))
design_and_product = json.load(open(bath_path + r'design-and-product.json', encoding='utf8'))
electrical_engineering = json.load(open(bath_path + r'electrical-engineering.json', encoding='utf8'))
math_and_logic = json.load(open(bath_path + r'math-and-logic.json', encoding='utf8'))
mobile_and_web_development = json.load(open(bath_path + r'mobile-and-web-development.json', encoding='utf8'))
networking = json.load(open(bath_path + r'networking.json', encoding='utf8'))
probability_and_statistics = json.load(open(bath_path + r'probability-and-statistics.json', encoding='utf8'))
security = json.load(open(bath_path + r'security.json', encoding='utf8'))
machine_learning = json.load(open(bath_path + r'machine-learning.json', encoding='utf8'))
data_analysis = json.load(open(bath_path + r'data-analysis.json', encoding='utf8'))
software_development = json.load(open(bath_path + r'software-development.json', encoding='utf8'))

all_data = computer_security_and_networks + \
algorithms + \
cloud_computing + \
data_management + \
design_and_product + \
electrical_engineering + \
math_and_logic + \
mobile_and_web_development + \
networking + \
probability_and_statistics + \
security + \
machine_learning + \
data_analysis + \
software_development

In [ ]:
len(all_data)

In [ ]:
title=[]; week_data=[]; target_audience=[]; created_by=[]; teach_by=[]
week_data_title=[]; course_info=[]; department=[]; week_data_desc=[]
package_num=[];package_name=[]; level=[]; rating=[]; index=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    for i in range(len(all_data[n]['teach_by'])):
            title.append(all_data[n]['title'])
            target_audience.append(all_data[n]['target_audience'])
            created_by.append(all_data[n]['created_by'])
            course_info.append(all_data[n]['course_info'])
            package_num.append(all_data[n]['package_num'])
            package_name.append(all_data[n]['package_name'])
            level.append(all_data[n]['level'])
            rating.append(all_data[n]['rating'])
            teach_by.append(list((all_data[n]['teach_by'][i].values()))[0])  
            department.append(list((all_data[n]['teach_by'][i].values()))[1])

In [ ]:
data = pd.DataFrame({'title': title,
                    'target_audience': target_audience,
                    'created_by' : created_by,  
                    'course_info': course_info,
                    'teach_by': teach_by,
                     'department': department,
                     'level': level,
                     'rating': rating,
                     'package_num': package_num,
                     'package_name': package_name          
                    },
                    columns = ['title', 'course_info', 'target_audience', 'created_by','teach_by',
                               'department','package_num','package_name', 'level','rating'])

In [ ]:
data

In [ ]:
data =  data.drop_duplicates(subset='title')

In [ ]:
for i, row in data.iterrows():
    if not isEnglish(row.title):
        data.drop(i, inplace=True)

In [ ]:
len(data.title.unique())

In [ ]:
title=[]; created_by=[]; course_info=[]
data_title=[]; data_desc=[]; data_video=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    week_data_title=[];  week_data_desc=[];week_data_video=[]
    title.append(all_data[n]['title'])
    created_by.append(all_data[n]['created_by'])
    for i in range(len(all_data[n]['week_data'])):
        week_data_desc.append(all_data[n]['week_data'][i]['description'])
        week_data_title.append(all_data[n]['week_data'][i]['title'])
        week_data_video += all_data[n]['week_data'][i]['video']
    data_title.append(week_data_title)
    data_desc.append(week_data_desc)
    data_video.append(week_data_video)

In [ ]:
data_created_by = pd.DataFrame({'title': title,
                    'created_by':created_by},
                    columns = ['title', 'created_by'])

data_detail = pd.DataFrame({     
                    'week_data_desc': data_desc,
                    'week_data_title': data_title,
                    'week_data_video': data_video
                    },
                    columns = ['week_data_desc', 'week_data_title','week_data_video'])


In [ ]:
data_created_by.drop_duplicates(subset= 'title', inplace=True)

In [ ]:
len(data_created_by)

In [ ]:
for i, row in data_created_by.iterrows():
    if not isEnglish(row.title):
        data_created_by.drop(i, inplace=True)

In [ ]:
data_detail = pd.merge(data_created_by, data_detail, left_index=True, right_index=True, how="left")

In [ ]:
data_detail.head()

In [ ]:
for i, row in data_detail.iterrows():
    if len(row.week_data_video) == 0:
        data_detail.drop(i, inplace=True)

In [ ]:
data_detail = data_detail.reset_index(drop=True)

In [ ]:
nlp = spacy.load('en')

In [ ]:
week_video_list = []
for video_data in data_detail.week_data_video:
    video_str = ''
    x = ' '.join(video_data)
    doc = nlp(x.lower())
    for token in doc:
        if not token.is_stop and token.is_alpha:
            video_str += token.lemma_
            video_str += ' '
    week_video_list.append(video_str)

In [ ]:
data_detail['week_data_video_str'] = pd.Series(week_video_list)
data_detail.dropna(inplace=True)
data_detail = data_detail.reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorize = TfidfVectorizer(max_df=150)
bows = vectorize.fit(data_detail.week_data_video_str)
document_matrix = bows.transform(data_detail.week_data_video_str).toarray()

In [ ]:
document_matrix.shape

In [ ]:
f = bows.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
result = cosine_similarity(document_matrix)

In [ ]:
data_detail.loc[np.argsort(result[20])[::-1], :]

In [ ]:
## build title & Top-5 relation

def get_nth_relation(uid, n_th):
    return data_detail.loc[np.argsort(result[uid])[::-1][n_th], :]['title']

title_relation_df = pd.DataFrame(data_detail['title'])
title_relation_df['UID'] = list(range(len(data_detail)))

top_n = 5
for i in range(1, top_n+1):
    str_i = str(i)
    title_relation_df['relate_'+str_i] = title_relation_df['UID'].apply(lambda x: get_nth_relation(x, i))

title_relation_df.head(5)

In [ ]:
df = pd.merge(title_relation_df, data, on='title', how='left')

In [ ]:
df.head()

In [ ]:
from py2neo import Graph

graph = Graph(password='pwd')

In [686]:
graph.delete_all()

In [687]:
from py2neo import Node
institute_dict = {}
for created_by in set(data.created_by):
    institute_dict[created_by] = Node("Institute", name=created_by)
    graph.create(institute_dict[created_by])

In [688]:
len(set(teach_dict))

491

In [689]:
teach_dict = {}
for teach_by in set(data.teach_by):
    teach_dict[teach_by] = Node("Teacher", name=teach_by)
    graph.create(teach_dict[teach_by])

In [690]:
from py2neo import Relationship
for i, row in data.iterrows():
    x = teach_dict[row.teach_by]
    y = institute_dict[row.created_by]
    graph.create(Relationship(x, "from", y))
    

In [693]:
course_dict = {}
for i, row in df.iterrows():
    course_dict[row.title] = Node("Course", name=row.title, rating=row.rating, level=row.level)
    graph.create(course_dict[row.title])

In [694]:
for i, row  in data.iterrows():
    if row.title in course_dict:
        x = teach_dict[row.teach_by]
        y = course_dict[row.title]
        graph.create(Relationship(x, "teach", y))
    
    

In [ ]:
data.group_by()

In [692]:
from vis import draw

options = {"Teacher": "name"}
draw(graph, options)



In [737]:
df = df.sort_values(by=['package_name', 'package_num'])


In [739]:
for name, group in df.groupby('package_name')['title']:
    tmp = list(group)
    for i, row in enumerate(tmp):
        if i != 0:
            x = course_dict[tmp[i-1]]
            y = course_dict[row]
            graph.create(Relationship(x, "require", y))
            

In [744]:
for name, group in df.groupby('package_name')['title']:
    tmp = list(group)
    for i, row in enumerate(tmp):
        if i != 0:
            x = course_dict[tmp[i-1]]
            y = course_dict[row]
            graph.create(Relationship(x, "require", y))

403

In [783]:
for i, row in title_relation_df.iterrows():
    title = row['title']
    for i in range(1, 6):
        x = course_dict[title]
        y = course_dict[row['relate_%s' % i]]
        if df[df.title == title].package_name.values[0] != None:
            if df[df.title == row['relate_%s' % i]].package_name.values[0] != df[df.title == title].package_name.values[0]:
                graph.create(Relationship(x, "similar", y))
        else:
            graph.create(Relationship(x, "similar", y))

In [748]:
title_relation_df.head()

,title,UID,relate_1,relate_2,relate_3,relate_4,relate_5
0,Bitcoin and Cryptocurrency Technologies,0,Software Design Threats and Mitigations,Cryptographic Hash and Integrity Protection,Process Mining: Data science in Action,What is a Proof?,Build Your First Android App (Project-Centered...
1,Cryptography I,1,Cryptography,Symmetric Cryptography,Basic Cryptography and Programming with Crypto...,Asymmetric Cryptography and Key Management,Introduction to Cybersecurity for Business
2,Google Cloud Platform Fundamentals: Core Infra...,2,Getting Started With Application Development,Develop and Deploy Windows Applications on Goo...,Elastic Cloud Infrastructure: Containers and S...,Essential Cloud Infrastructure: Core Services,Google Cloud Platform Fundamentals for AWS Pro...
3,G Suite Administrator Fundamentals,3,Introduction to Systems Engineering,The Unix Workbench,Econometrics: Methods and Applications,Front-End Web UI Frameworks and Tools: Bootstr...,Introduction to Logic
4,Usable Security,4,Data Science Ethics,"Games, Sensors and Media","Privacy, Reputation, and Identity in a Digital...",Introduction to UI Design,Securing Digital Democracy


In [769]:
df[-10:-1]

,index,title,UID,relate_1,relate_2,relate_3,relate_4,relate_5,course_info,target_audience,created_by,teach_by,department,package_num,package_name,level,rating
585,564,Learn to Program: Crafting Quality Code,564,Algorithmic Toolbox,Programming Fundamentals,"Divide and Conquer, Sorting and Searching, and...",Algorithmic Thinking (Part 2),"Graph Search, Shortest Paths, and Data Structures",About this course: Not all programs are create...,None,University of Toronto,Jennifer Campbell,Department of Computer Science,None,None,None,4.5
586,566,Audio Signal Processing for Music Applications,566,Digital Signal Processing,Creative Programming for Digital Media & Mobil...,Fundamentals of Audio and Music Engineering: P...,3D Models for Virtual Reality,Responsive Website Tutorial and Examples,About this course: In this course you will lea...,Who is this class for: This course is primary ...,"Universitat Pompeu Fabra of Barcelona, Stanfor...",Xavier Serra,Dept. of Information and Communication Technol...,None,None,Intermediate,4.9
587,567,A Developer's guide to Node-RED,567,A developer's guide to the Internet of Things ...,"Server-side Development with NodeJS, Express a...",Accounting Analytics,Essential Cloud Infrastructure: Core Services,"Graph Search, Shortest Paths, and Data Structures",About this course: >>> By enrolling in this co...,Who is this class for: The course is aimed at ...,IBM,Brian Innes,IBM Digital Business Group,None,None,Intermediate,None
588,572,Build a Modern Computer from First Principles:...,572,Build a Modern Computer from First Principles:...,Game Theory II: Advanced Applications,iOS App Development Basics,Business Statistics and Analysis Capstone,Developing Android Apps with App Inventor,About this course: In this project-centered co...,Who is this class for: Do you want to understa...,Hebrew University of Jerusalem,Shimon Schocken,Computer Science,None,None,None,4.9
589,573,"C++ For C Programmers, Part B",573,"C++ For C Programmers, Part A","Programming Languages, Part A",Introduction to Programming with MATLAB,Cloud Computing Concepts: Part 2,Calculus: Single Variable Part 1 - Functions,About this course: This course is for experien...,None,"University of California, Santa Cruz",Ira Pohl,Computer Science,None,None,None,4.2
590,577,Fundamentals of Parallelism on Intel Architecture,577,Global Warming II: Create Your Own Models in P...,Interactivity with JavaScript,Introduction to Embedded Systems Software and ...,"Responsive Website Basics: Code with HTML, CSS...",Fundamentals of Digital Image and Video Proces...,About this course: This course will introduce ...,Who is this class for: For developers of machi...,Intel,Andrey Vladimirov,Colfax International,None,None,Intermediate,2.0
591,579,Creative Programming for Digital Media & Mobil...,579,Approximation Algorithms Part I,Python for Genomic Data Science,The Raspberry Pi Platform and Python Programmi...,Functional Programming Principles in Scala,Dynamical Modeling Methods for Systems Biology,About this course: This course is for anyone w...,None,"University of London, Goldsmiths, University o...",Dr Marco Gillies,"Computing Department, Goldsmiths, University o...",None,None,None,4.7
592,580,Software Architecture for the Internet of Things,580,Software Development Processes and Methodologies,Requirements Gathering for Secure Software Dev...,Software Architecture,Design Patterns,Software Processes and Agile Practices,About this course: This course will teach you ...,Who is this class for: You're at the right add...,EIT Digital,Frank Gielen,UGent/IBCN: Internet Based Communication Networks,None,None,None,4.2
593,584,Quantitative Model Checking,584,An Intuitive Introduction to Probability,Social Media Data Analytics,Data-driven Astronomy,"Designing, Running, and Analyzing Experiments",Control of Nonlinear Spacecraft Attitude Motion,About this course: The integration of ICT (inf...,Who is this class for: This course is meant fo...,EIT Digital,Anne Remke,Computer Scien

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

svd = TruncatedSVD(100)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X = lsa.fit_transform(document_matrix)
inertia_list = []  
for i in range(2,15):
    kmeans = KMeans(n_clusters=i).fit(X)
    inertia_list.append(kmeans.inertia_)

import matplotlib.pyplot as plt
plt.plot(range(2,15), inertia_list, linestyle='-', marker='o')
plt.show() 

In [ ]:
![image.png](attachment:image.png)

In [ ]:
#IsDuplicated = data_to_mysql.duplicated()

In [ ]:
# duplicate_course=0
# for i in range(len(data_to_mysql)):
#     if IsDuplicated[i] == True:
#         duplicate_course=duplicate_course+1
#         #print(i)
# print(duplicate_course)

In [ ]:
#data = data_to_mysql.drop_duplicates()

In [ ]:
#data_to_mysql.head()

### --------------------------------------------------------------------------------

In [ ]:
### Testing

In [ ]:
# v=[]
# for n in range(len(all_data)):
#     for j in range(len(all_data[n]['week_data'])):     
#         v.append(list((all_data[n]['week_data'][j]).values())[0])
#         #a.append(pd.Series(v).str.cat(sep='')) 

In [ ]:
# v= pd.Series(v).str.cat(sep='')